In [ ]:
import torch 
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
torch.manual_seed(777)

In [ ]:
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100
#배치 크기는 보통 2의 제곱수를 사용합니다. ex) 2, 4, 8, 16, 32, 64... 그 이유는 CPU와 GPU의 메모리가 2의 배수이므로 배치크기가 2의 제곱수일 경우에 데이터 송수신의 효율을 높일 수 있다고 합니다

In [ ]:
mnist_train = dsets.MNIST(root = 'MNIST_data/',train = True, transform = transforms.ToTensor(),download = True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',train = False, transform = transforms.ToTensor(), download = True)
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

In [ ]:
class CNN(torch.nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.keep_prob = 0.5
    # 이미지 처리의 기본적인 용어 높이 : 이미지의 세로방향 픽셀, 너비 : 이미지의 가로바양 픽셀
    # 채널 : 이미지의 색 성분이다. 흑백이미지 이면 채널값은 1이다
                  # 합성곱 연산 메소드 1채널 짜리를 입력받아 32채널을 결과값을 생성, 커 널    
    self.layer1 = torch.nn.Sequential(torch.nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
                                     torch.nn.ReLU(),torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
                  # 32채널을 입력받아 64 채널의  결과값 생성
    self.layer2 = torch.nn.Sequential(torch.nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
                                     torch.nn.ReLU(),torch.nn.MaxPool2d(kernel_size = 2, stride = 2))
                  # 64채널을 입력받아 128 채널의 결과값 생성
    self.layer3 = torch.nn.Sequential(torch.nn.Conv2d(64,128, kernel_size = 3, stride = 1, padding = 1),
                                     torch.nn.ReLU(),torch.nn.MaxPool2d(kernel_size = 2, stride=2, padding = 1))
    #self.layer3 = torch.nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)

    #self.fc =  torch.nn.Linear(7*7*64, 10, bias = True)
    # ??
    self.fc1 =  torch.nn.Linear(4*4*128, 625, bias = True) # fully connected 선언 
    torch.nn.init.xavier_uniform_(self.fc1.weight)
    # fully connected 다음 
    self.layer4 = torch.nn.Sequential(
            self.fc1, # fulltconnected 실행
            torch.nn.ReLU(),
            torch.nn.Dropout(p=1 - self.keep_prob))# 오버피팅을 방지하기 위해 학습과정에서 신경망의 일부를 사용하지 않느다
            # 0.5 이면 절반만 사용함
    # L5 Final FC 625 inputs -> 10 outputs
    self.fc2 = torch.nn.Linear(625, 10, bias=True) # 생성된 625개의 채널을 최종 출력(1~10) 만큼 변환
    torch.nn.init.xavier_uniform_(self.fc2.weight)
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = self.layer3(out)
    out = out.view(out.size(0), -1)
    out = self.layer4(out)
    out = self.fc2(out)
    return out

In [ ]:
model = CNN().to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
total_batch = len(data_loader)
print('총 배치의 수 : {}' .format(total_batch))
for epoch in range(training_epochs):
  avg_cost = 0
  for X,Y in data_loader:
    X = X.to(device)
    Y = Y.to(device)

    optimizer.zero_grad()
    hypothesis = model(X)
    cost = criterion(hypothesis, Y)
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch
print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))




In [ ]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())